<a href="https://colab.research.google.com/github/sskunkworks/ai-notebook/blob/main/text_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import os, h5py, numpy as np

from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, InputLayer
from keras.layers import Activation, Dropout, Flatten, Dense, Concatenate
from keras.layers import Input, Dense
from keras.layers.core import Activation
from keras.models import Model
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, merge
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras


In [27]:
class text_binary_crossentropy:
    def __getattr__(self, name):
        if name == '__name__':
            return 'text_binary_crossentropy'
        raise AttributeError

    def __call__(self, y_true, y_pred):
        return K.mean(K.binary_crossentropy(y_pred, y_true), axis=1)


In [ ]:
vgg16_weights_path = 'drive/MyDrive/weights/vgg16_weights.h5'

def pretrained_vgg16():
  
  model = Sequential()
  model.add(InputLayer(input_shape=(224,224,3)))
  model.add(ZeroPadding2D(padding=(1, 1), name='zero1_1'))
  model.add(Conv2D(filters=64, kernel_size=3, name='conv1_1'))
  model.add(ZeroPadding2D(padding=(1, 1), name='zero1_2'))
  model.add(Conv2D(filters=64, kernel_size=3, name='conv1_2'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='max1_1'))

  model.add(ZeroPadding2D(padding=(1, 1), name='zero2_1'))
  model.add(Conv2D(filters=128, kernel_size=3, name='conv2_1'))
  model.add(ZeroPadding2D(padding=(1, 1), name='zero2_2'))
  model.add(Conv2D(filters=128, kernel_size=3, name='conv2_2'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='max2_1'))

  model.add(ZeroPadding2D(padding=(1, 1), name='zero3_1'))
  model.add(Conv2D(filters=256, kernel_size=3, name='conv3_1'))
  model.add(ZeroPadding2D(padding=(1, 1), name='zero3_2'))
  model.add(Conv2D(filters=256, kernel_size=3, name='conv3_2'))
  model.add(ZeroPadding2D(padding=(1, 1), name='zero3_3'))
  model.add(Conv2D(filters=256, kernel_size=3, name='conv3_3'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='max3_1'))

  model.add(ZeroPadding2D(padding=(1, 1), name='zero4_1'))
  model.add(Conv2D(filters=512, kernel_size=3, name='conv4_1'))
  model.add(ZeroPadding2D(padding=(1, 1), name='zero4_2'))
  model.add(Conv2D(filters=512, kernel_size=3, name='conv4_2'))
  model.add(ZeroPadding2D(padding=(1, 1), name='zero4_3'))
  model.add(Conv2D(filters=512, kernel_size=3, name='conv4_3'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='max4_1'))

  model.add(ZeroPadding2D(padding=(1, 1), name='zero5_1'))
  model.add(Conv2D(filters=512, kernel_size=3, name='conv5_1'))
  model.add(ZeroPadding2D(padding=(1, 1), name='zero5_2'))
  model.add(Conv2D(filters=512, kernel_size=3, name='conv5_2'))
  model.add(ZeroPadding2D(padding=(1, 1), name='zero5_3'))
  model.add(Conv2D(filters=512, kernel_size=3, name='conv5_3'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='max5_1'))

  # model.summary()
    
  f = h5py.File(vgg16_weights_path)
  for k in range(f.attrs['nb_layers']):
    if k >= len(model.layers):
      break
      g = f['layer_{}'.format(k)]
      weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
      model.layers[k].set_weights(weights)
      model.layers[k].trainable = False
  f.close()
  
  conv_dict = dict([(layer.name, layer)
      for layer in model.layers if isinstance(layer, Convolution2D)])
  return model, conv_dict

pre_model = pretrained_vgg16()


In [40]:
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense
from tensorflow.keras import Model

def my_vgg16():
  input = Input(shape =(224,224,3))
  # 1st Conv Block
  x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu', name='conv1_1')(input)
  x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu', name='conv1_2')(x)
  x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
  # 2nd Conv Block
  x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu', name='conv2_1')(x)
  x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu', name='conv2_2')(x)
  x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
  # 3rd Conv block  
  x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu', name='conv3_1')(x) 
  x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu', name='conv3_2')(x) 
  x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu', name='conv3_3')(x) 
  x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
  # 4th Conv block
  x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu', name='conv4_1')(x)
  x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu', name='conv4_2')(x)
  x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu', name='conv4_3')(x)
  x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
  # 5th Conv block
  x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu', name='conv5_1')(x)
  x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu', name='conv5_2')(x)
  x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu', name='conv5_3')(x)
  x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
  
  return Model(inputs=input, outputs =x)

base_model = my_vgg16()

#이 층들의 활성화를 이용합시다
layer_names = [
    'conv1_2', 
    'conv2_2',
    'conv3_3',   
    'conv4_3',   
    'conv5_3',   
]
layers = [base_model.get_layer(name).output for name in layer_names]

down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)
down_stack.trainable = False

# inputs = tf.keras.layers.Input(shape=[224, 224, 3])
# x = inputs
# # 모델을 통해 다운샘플링합시다
# skips = down_stack(x)

# # s1 = Conv2D(filters=1, kernel_size=1, name='stage1')(x1)
# # s2 = Conv2D(filters=1, kernel_size=1, name='stage2')(x2)
# # s3 = Conv2D(filters=1, kernel_size=1, name='stage3')(x3)
# # s4 = Conv2D(filters=1, kernel_size=1, name='stage4')(x4)
# # s5 = Conv2D(filters=1, kernel_size=1, name='stage5')(x5)

conv1_2 = down_stack.get_layer('conv1_2').output
conv2_2 = down_stack.get_layer('conv2_2').output
conv3_3 = down_stack.get_layer('conv3_3').output
conv4_3 = down_stack.get_layer('conv4_3').output
conv5_3 = down_stack.get_layer('conv5_3').output


s1 = Conv2D(filters=1, kernel_size=1, name='stage1')(conv1_2)
s2 = Conv2D(filters=1, kernel_size=1, name='stage2')(conv2_2)
s3 = Conv2D(filters=1, kernel_size=1, name='stage3')(conv3_3)
s4 = Conv2D(filters=1, kernel_size=1, name='stage4')(conv4_3)
s5 = Conv2D(filters=1, kernel_size=1, name='stage5')(conv5_3)

y = UpSampling2D((2,2))(s5)
y = UpSampling2D((2,2))(y)
y = UpSampling2D((2,2))(y)
y5 = UpSampling2D((2,2))(y)

y = UpSampling2D((2,2))(s4)
y = UpSampling2D((2,2))(y)
y4 = UpSampling2D((2,2))(y)

y = UpSampling2D((2,2))(s3)
y3 = UpSampling2D((2,2))(y)

y2 = UpSampling2D((2,2))(s2)

y1 = s1

concat = Concatenate()
m = concat([y5, y4])
m = concat([m, y3])
m = concat([m, y2])
m = concat([m, y1])
m = Conv2D(filters=2, kernel_size=1, activation='sigmoid')(m)

model = tf.keras.Model(inputs=down_stack.input, outputs=m)

for layer in model.layers[:6]:
    layer.trainable = False

model.compile(optimizer=optimizers.SGD(learning_rate=1e-4, momentum=0.9), loss=text_binary_crossentropy(), metrics=['accuracy'])

# model.summary()

In [ ]:
#model = textFCN_block()
tf.keras.utils.plot_model(model, show_shapes=True)

In [41]:
model.load_weights("drive/MyDrive/weights/text_weights.h5")

ValueError: ignored